## TF-IDF 알고리즘
- 예제 : https://wikidocs.net/24603

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import os
print(os.listdir("../content/drive/MyDrive/추천시스템/archive/movies"))

['movies_metadata.csv', 'ratings.csv']


In [4]:
# 경로의 경우 각자의 환경에 맞게 설정해주면 됩니다. 
path = '../content/drive/MyDrive/추천시스템/archive/movies/'

In [5]:
data = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [6]:
data['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [7]:
# overview의 항목 추출 
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [8]:
data['overview'].notnull()

0        True
1        True
2        True
3        True
4        True
         ... 
45461    True
45462    True
45463    True
45464    True
45465    True
Name: overview, Length: 45466, dtype: bool

In [9]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

(44512, 24)

In [10]:
data = data.loc[0:10000].reset_index(drop=True)

In [11]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
# https://wikidocs.net/22530
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(10001, 10000)


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 만일 여기서 메모리 에러가 발생하신 분은 TF-IDF의 파라미터를 수정해줘서 다시 돌리면 됩니다. 
# tfidf = TfidfVectorizer(stop_words='english', max_features=10000)

# 그래도, 안되는 경우에는 문서의 수를 조금 줄여서 실행해보시길 바랍니다. 
# data = data.loc[0:10000].reset_index(drop=True)

In [13]:
cosine_matrix.shape

(10001, 10001)

In [14]:
np.round(cosine_matrix, 4)

array([[1.    , 0.018 , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.018 , 1.    , 0.0559, ..., 0.    , 0.    , 0.    ],
       [0.    , 0.0559, 1.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 1.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 1.    ]])

In [ ]:
# cosine_matrix
# 인덱스 0 -> 1 -> 2 ... -> 10001
# 0 인덱스 : 0번째 id 영화

In [18]:
for i,c in enumerate(data['title']):
  print(i)
  print(c)
  break

0
Toy Story


In [15]:
# movie title와 id를 매핑할 dictionary를 생성해줍니다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해줍니다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [19]:
movie2id

{0: 'Toy Story',
 1: 'Jumanji',
 2: 'Grumpier Old Men',
 3: 'Waiting to Exhale',
 4: 'Father of the Bride Part II',
 5: 'Heat',
 6: 'Sabrina',
 7: 'Tom and Huck',
 8: 'Sudden Death',
 9: 'GoldenEye',
 10: 'The American President',
 11: 'Dracula: Dead and Loving It',
 12: 'Balto',
 13: 'Nixon',
 14: 'Cutthroat Island',
 15: 'Casino',
 16: 'Sense and Sensibility',
 17: 'Four Rooms',
 18: 'Ace Ventura: When Nature Calls',
 19: 'Money Train',
 20: 'Get Shorty',
 21: 'Copycat',
 22: 'Assassins',
 23: 'Powder',
 24: 'Leaving Las Vegas',
 25: 'Othello',
 26: 'Now and Then',
 27: 'Persuasion',
 28: 'The City of Lost Children',
 29: 'Shanghai Triad',
 30: 'Dangerous Minds',
 31: 'Twelve Monkeys',
 32: 'Babe',
 33: 'Carrington',
 34: 'Dead Man Walking',
 35: 'Across the Sea of Time',
 36: 'It Takes Two',
 37: 'Clueless',
 38: 'Cry, the Beloved Country',
 39: 'Richard III',
 40: 'Dead Presidents',
 41: 'Restoration',
 42: 'Mortal Kombat',
 43: 'To Die For',
 44: 'How To Make An American Quilt',
 

In [20]:
id2movie

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 5,
 'Sabrina': 876,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'The American President': 10,
 'Dracula: Dead and Loving It': 11,
 'Balto': 12,
 'Nixon': 13,
 'Cutthroat Island': 14,
 'Casino': 15,
 'Sense and Sensibility': 16,
 'Four Rooms': 17,
 'Ace Ventura: When Nature Calls': 18,
 'Money Train': 19,
 'Get Shorty': 20,
 'Copycat': 21,
 'Assassins': 22,
 'Powder': 23,
 'Leaving Las Vegas': 24,
 'Othello': 2715,
 'Now and Then': 26,
 'Persuasion': 27,
 'The City of Lost Children': 28,
 'Shanghai Triad': 29,
 'Dangerous Minds': 30,
 'Twelve Monkeys': 31,
 'Babe': 32,
 'Carrington': 33,
 'Dead Man Walking': 34,
 'Across the Sea of Time': 35,
 'It Takes Two': 36,
 'Clueless': 37,
 'Cry, the Beloved Country': 38,
 'Richard III': 7161,
 'Dead Presidents': 40,
 'Restoration': 41,
 'Mortal Kombat': 42,
 'To Die For': 43,
 'How To Make An American Quilt':

In [21]:
movie2id.items()

dict_items([(0, 'Toy Story'), (1, 'Jumanji'), (2, 'Grumpier Old Men'), (3, 'Waiting to Exhale'), (4, 'Father of the Bride Part II'), (5, 'Heat'), (6, 'Sabrina'), (7, 'Tom and Huck'), (8, 'Sudden Death'), (9, 'GoldenEye'), (10, 'The American President'), (11, 'Dracula: Dead and Loving It'), (12, 'Balto'), (13, 'Nixon'), (14, 'Cutthroat Island'), (15, 'Casino'), (16, 'Sense and Sensibility'), (17, 'Four Rooms'), (18, 'Ace Ventura: When Nature Calls'), (19, 'Money Train'), (20, 'Get Shorty'), (21, 'Copycat'), (22, 'Assassins'), (23, 'Powder'), (24, 'Leaving Las Vegas'), (25, 'Othello'), (26, 'Now and Then'), (27, 'Persuasion'), (28, 'The City of Lost Children'), (29, 'Shanghai Triad'), (30, 'Dangerous Minds'), (31, 'Twelve Monkeys'), (32, 'Babe'), (33, 'Carrington'), (34, 'Dead Man Walking'), (35, 'Across the Sea of Time'), (36, 'It Takes Two'), (37, 'Clueless'), (38, 'Cry, the Beloved Country'), (39, 'Richard III'), (40, 'Dead Presidents'), (41, 'Restoration'), (42, 'Mortal Kombat'), (43

In [16]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(2979, 0.44592327052916614),
 (8303, 0.2233391183443228),
 (6920, 0.21080722391751464),
 (5776, 0.20197550698885142),
 (1058, 0.1935716526306723),
 (1916, 0.17059971408350388),
 (5597, 0.16763967874757163),
 (483, 0.16369854388559427),
 (3039, 0.15050720419194555),
 (7591, 0.14809453112651405)]

In [17]:
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 2', 0.44592327052916614),
 ('The Champ', 0.2233391183443228),
 ('Rivers and Tides', 0.21080722391751464),
 ('Class of 1984', 0.20197550698885142),
 ('Rebel Without a Cause', 0.1935716526306723),
 ('Condorman', 0.17059971408350388),
 ('Heartbeeps', 0.16763967874757163),
 ('Malice', 0.16369854388559427),
 ('Man on the Moon', 0.15050720419194555),
 ('The First $20 Million Is Always the Hardest', 0.14809453112651405)]